In [1]:
import os
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import datasets
from collections import Counter

from joblib import dump
from sklearn.externals import joblib

#from keras.models import model_from_json

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


C:\Users\KaiLo\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Se importa el Dataset sin informacion de nubes

In [2]:
path = './satellite_dataset-con-nubes.csv'

satellite = pd.read_csv(path)
satellite.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,cloud,ninguno
0,286.0,197.0,96.0,83.0,116.0,95.0,1,0,0,0,0
1,280.0,217.0,108.0,83.0,118.0,96.0,1,0,0,0,0
2,3011.0,2996.0,2900.0,4186.0,1683.0,1373.0,0,0,0,1,0
3,3576.0,3303.0,3313.0,3369.0,2637.0,2339.0,0,0,0,1,0
4,1398.0,1345.0,1311.0,1389.0,1633.0,1355.0,0,0,0,0,1


# Filtro de valores negativos
Se realiza una filtro de las columnas que en alguna banda contengan un valor negativo

In [3]:
satellite = satellite[satellite["blue"] > 0]
satellite = satellite[satellite["red"] > 0]
satellite = satellite[satellite["green"] > 0]
satellite = satellite[satellite["nir"] > 0]
satellite = satellite[satellite["swir1"] > 0]
satellite = satellite[satellite["swir2"] > 0]

In [4]:
#satellite['wofs'].value_counts()
#satellite['bosque'].value_counts()
#satellite['wofs_bosque'].value_counts()
#satellite['cloud'].value_counts()
#satellite['ninguno'].value_counts()

# Se reemplaza los valores en columnas de clases
Como se puede observar el dataset, las columnas de las clases se encuentran categorizadas por un valor binario. Lo que se realiza a continuación es reemplazar estos valores binarios por NaN en caso de ser "Cero" y el nombre de la clase en caso de ser "Uno".

In [6]:
# Reemplazar el valor de columna Wofs por NaN(0) y Wofs(1)
satellite['wofs'] = satellite['wofs'].replace(0,np.nan)
satellite['wofs'] = satellite['wofs'].replace(1,'wofs')


In [7]:
# Reemplazar el valor de columna bosque por NaN(0) y bosque(1)
satellite['bosque'] = satellite['bosque'].replace(0,np.nan)
satellite['bosque'] = satellite['bosque'].replace(1,'bosque')

In [8]:
# Reemplazar el valor de columna wofs_bosque por NaN(0) y wofs_bosque(1)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(0,np.nan)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(1,'wofs_bosque')

In [9]:
# Reemplazar el valor de columna ninguno por NaN(0) y ninguno(1)
satellite['ninguno'] = satellite['ninguno'].replace(0,np.nan)
satellite['ninguno'] = satellite['ninguno'].replace(1,'ninguno')

In [10]:
# Reemplazar el valor de columna ninguno por NaN(0) y cloud(1)
satellite['cloud'] = satellite['cloud'].replace(0,np.nan)
satellite['cloud'] = satellite['cloud'].replace(1,'cloud')

# Nueva columna de clasificación
En vista que debemos tener la clasificación en una sola columna. Procedemos a crearla con el nombre "label" y alli asignaremos el valor categorizado de cada conjunto de atributos.

In [11]:
#Se crea una nueva columna para almacenar el valor de etiqueta
satellite['label']=np.nan

#Se indica en la columna 'label', la clasificacion de la columna 'wofs'. Se esribe "Wofs" en columna 'label' si en  
#la columna 'wofs' esta este valor
for label in [col for col in satellite.columns if 'wofs' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'bosque'. Se esribe "bosque" en columna 'label' si en  
#la columna 'bosque' esta este valor
for label in [col for col in satellite.columns if 'bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'wofs_bosque'. Se esribe "wofs_bosque" en columna 'label' si en  
#la columna 'wofs_bosque' esta este valor
for label in [col for col in satellite.columns if 'wofs_bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'ninguno'. Se esribe "ninguno" en columna 'label' si en  
#la columna 'ninguno' esta este valor
for label in [col for col in satellite.columns if 'ninguno' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'cloud'. Se esribe "cloud" en columna 'label' si en  
#la columna 'cloud' esta este valor
for label in [col for col in satellite.columns if 'cloud' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
satellite.head(20)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,cloud,ninguno,label
0,286.0,197.0,96.0,83.0,116.0,95.0,wofs,NaN,NaN,NaN,NaN,wofs
1,280.0,217.0,108.0,83.0,118.0,96.0,wofs,NaN,NaN,NaN,NaN,wofs
2,3011.0,2996.0,2900.0,4186.0,1683.0,1373.0,NaN,NaN,NaN,cloud,NaN,cloud
3,3576.0,3303.0,3313.0,3369.0,2637.0,2339.0,NaN,NaN,NaN,cloud,NaN,cloud
4,1398.0,1345.0,1311.0,1389.0,1633.0,1355.0,NaN,NaN,NaN,NaN,ninguno,ninguno
5,328.0,232.0,118.0,96.0,124.0,95.0,wofs,NaN,NaN,NaN,NaN,wofs
6,230.0,203.0,105.0,96.0,129.0,105.0,wofs,NaN,NaN,NaN,NaN,wofs
7,218.0,317.0,142.0,125.0,169.0,150.0,wofs,NaN,NaN,NaN,NaN,wofs
8,6702.0,6782.0,7018.0,7304.0,6323.0,5154.0,NaN,NaN,NaN,cloud,NaN,cloud
9,2866.0,2807.0,2806.0,2901.0,3267.0,2668.0,NaN,NaN,NaN,cloud,NaN,cloud


In [13]:
# Se crea el conjunto de clasificacion y el de entrenamiento
Y = satellite["label"]
X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","cloud","ninguno"], axis=1)
Y.value_counts()

cloud          538313
wofs           288423
bosque          99236
ninguno         61378
wofs_bosque       300
Name: label, dtype: int64

# Se crean los conjuntos de entrenamiento y de prueba
Para hacer esto se deben dividir los datos en 4 conjuntos de datos de la siguiente manera: Conjuntos de entrenamiento(X_train, y_train) y Conjunto de test(X_test, y_test)

In [14]:
# Se dividen los datos en los conjuntos de Train y test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=0
)

In [15]:
X_test

,blue,green,red,nir,swir1,swir2
441018,4415.0,4694.0,4728.0,4888.0,4886.0,4312.0
738664,3464.0,3533.0,3523.0,4645.0,3161.0,2128.0
472244,393.0,334.0,238.0,220.0,319.0,257.0
574853,5000.0,5063.0,5114.0,5233.0,5074.0,4151.0
434050,2950.0,2903.0,2836.0,3711.0,2514.0,1761.0
728385,212.0,377.0,214.0,161.0,127.0,105.0
177393,154.0,438.0,219.0,3341.0,1372.0,484.0
848812,1212.0,1279.0,1127.0,3254.0,1728.0,1051.0
489892,5699.0,5788.0,5813.0,5858.0,5049.0,3801.0
952757,7983.0,8574.0,8785.0,9480.0,6711.0,4588.0


In [16]:
y_test

441018      cloud
738664      cloud
472244       wofs
574853      cloud
434050      cloud
728385       wofs
177393     bosque
848812      cloud
489892      cloud
952757      cloud
524790      cloud
889190      cloud
871177       wofs
401322      cloud
17962     ninguno
226096      cloud
904345       wofs
250812      cloud
396424      cloud
796464      cloud
971028      cloud
396610      cloud
115535      cloud
592753      cloud
961986      cloud
947227      cloud
593843      cloud
637936       wofs
819197      cloud
610996     bosque
           ...   
879536      cloud
403125      cloud
278461       wofs
244099       wofs
745752      cloud
927116     bosque
759299       wofs
327497       wofs
700841      cloud
889866      cloud
470887      cloud
539041      cloud
716410      cloud
17805        wofs
827759      cloud
717575      cloud
440067       wofs
931976      cloud
266738      cloud
818441      cloud
917705       wofs
935183      cloud
7543        cloud
609188     bosque
73642     

# Naive Bayes para multiples capas
La clasificación multicapa del algoritmo Naive Bayes es conocida como "Clasificación Naive Bayes Multinomial". Como se puede apreciar la clasificación de nuestro conjunto de caracteristicas puede tener 4 valores: wofs, bosque, wofs_bosqu y ninguno.

In [17]:
#Create a Gaussian Classifier
#gnb = GaussianNB()
gnb = MultinomialNB()

In [18]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [20]:
Counter(y_pred)

Counter({'ninguno': 45250,
         'cloud': 133091,
         'wofs': 83475,
         'bosque': 30659,
         'wofs_bosque': 3820})

In [21]:
confusion_matrix(y_test, y_pred)

array([[ 28721,      1,    320,      2,    551],
       [    43, 120350,  39851,    240,   1295],
       [  1895,   8902,   4897,   1003,   1646],
       [     0,   3826,    175,  82221,    278],
       [     0,     12,      7,      9,     50]], dtype=int64)

In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.94      0.97      0.95     29595
       cloud       0.90      0.74      0.82    161779
     ninguno       0.11      0.27      0.15     18343
        wofs       0.98      0.95      0.97     86500
 wofs_bosque       0.01      0.64      0.03        78

    accuracy                           0.80    296295
   macro avg       0.59      0.71      0.58    296295
weighted avg       0.88      0.80      0.83    296295



In [23]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7973101132317454


In [24]:
#toBePersisted = dict({
#    'metadata': {
#        'name': 'NB_Multinomial'
#    }
#})
#dump(toBePersisted, 'NB_Multinomial.joblib')

In [25]:
# joblib.dump(gnb, 'NB_Multinomial.joblib') 
joblib.dump(gnb, 'model_nb_multinomial.joblib') 

['model_nb_multinomial.joblib']

## Conclusiones



# Referencias
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

https://datascience.stackexchange.com/questions/40345/how-to-convert-multiple-columns-into-single-columns-in-pandas

https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra